In [3]:
!pip install transformers -q
!pip install wandb -q


     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 3.3MB 38.2MB/s 
     |████████████████████████████████| 901kB 39.3MB/s 
     |████████████████████████████████| 2.1MB 4.2MB/s 
     |████████████████████████████████| 133kB 39.5MB/s 
     |████████████████████████████████| 163kB 37.3MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 


In [4]:
import pandas as pd
import numpy as np
import torch 
from torch.utils.data import DataLoader , Dataset ,RandomSampler,SequentialSampler  
from transformers import T5Tokenizer , T5ForConditionalGeneration
import wandb

In [ ]:
#about data

# News Summary dataset available at Kaggle
# There are4514 rows of data. 
# Where each row has the following data-point:
# author : Author of the article
# date : Date the article was published
# headline: Headline for the published article
# read_more : URL for the article to follow online
# text: This is the summary of the article
# ctext: This is the complete article


In [1]:
!nvidia-smi
device   = "cuda"

Sun Apr 25 13:23:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!wandb login

wandb: Currently logged in as: husnain-ali21 (use `wandb login --relogin` to force relogin)


In [ ]:
class DataSet(Dataset):
    def __init__(self , df , tokenizer ,src_len , sum_len ):
        super(DataSet ,self).__init__()
        self.data = df
        self.tokenizer  = tokenizer
        self.src_len = src_len
        self.summary_len = sum_len
        self.summaries = self.data.text 
        self.articles = self.data.ctext

        def __len__(self):
            return len(self.summary)

        def __getitem__(self , index):
            article = str(self.articles[index])
            article = " ".join(articles.split())
            summary = str(self.summaries[index])
            summary = " ".join(summary.split())
            #The tokenizer uses the batch_encode_plus method to perform tokenization and generate
            # the necessary outputs, namely:source_id, source_mask 
            #from the actual text and target_id and target_mask from the summary text.

            src = self.tokenizer.batch_encode_plus([article]\
                    ,max_length = self.src_len , pad_to_max_len = True ,return_tensors ="pt")
            trg = self.tokenizer.batch_encode_plus([summary]\
                    ,max_length = self.summary_len , pad_to_max_len = True ,return_tensors ="pt")
            
            source_ids = src["input_ids"].squeeze()
            target_ids = trg["input_ids"].squeeze()

            source_masks = src["attention_mask"].squeeze()
            target_masks = trg["attention_mask"].squeeze()

            return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_masks': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)

            }



